# Run a multimodel comparison in a supercomputer holding the data pool

### The goal
We will plot the global mean annual mean near-surface air temperature for some climate model results as an example of a data-near server-side analysis. 

### The data
We will use the simulations of the Couple Model Intercomparison Project [CMIP6](https://pcmdi.llnl.gov/CMIP6/). We will concatenate historical data (that is, model results from 1850 to 2014 using the best estimates for anthropogenic and natural forcing) with the model results for the Shared Socioeconomic Pathway SSP2-4.5 based scenario (which corresponds to the growth in radiative forcing reached by 2100, in this case, 4.5 W/m2 or ~650 ppm CO2 equivalent). CMIP6 comprises several kind of experiments with various simulation members, and we will use some of them. You can find more information in the [CMIP6 Model and Experiment Documentation](https://pcmdi.llnl.gov/CMIP6/Guide/dataUsers.html#5-model-and-experiment-documentation).

### The software
We will use Python 3 [Pandas](https://pandas.pydata.org/), the popular data analysis package focused on labelled tabular data,  and [Xarray](http://xarray.pydata.org/), the Pandas generalization for n-dimensional arrays particularly tailored to working with NetCDF files, to process the data, together with the Climate Data Operators [cdo](https://pypi.org/project/cdo/) package.

### Where this analysis will happen or what data-near server-side means
This Jupyter notebook is meant to run in supercomputer [Mistral](https://www.dkrz.de/systems/hpc/hlre-3-mistral?set_language=en&cl=en) at the German Climate Computing Center (DKRZ) within the DKRZ [Jupyterhub](<https://jupyterhub.dkrz.de>) server. See in this [video](https://youtu.be/f0wZX9i0uWQ) the main features of the DKRZ Jupterhub/lab and how to use it. The DKRZ software developers made that all the required packages to run this Jupyter notebook are available under the Python 3 unstable kernel (see the top right corner of this page, you can also choose the kernel in the Kernel tab above), it contains all the common packages for geoscience applications. The DKRZ data managers made that the data pool is directly accessible (no need to download or transfer data, that is what "data-near" means) because DKRZ is also a node of the Earth System Grid Federation [ESGF](https://esgf.llnl.gov/) hosting more than 4 petabytes of CMIP6 data. More info about the data pool [here](https://www.dkrz.de/up/services/data-management/cmip-data-pool). Direct access to the data pool is one of the main benefits of the server-side data-near computing we demonstrate in this notebook.

The fist step is to [open an account in Mistral](https://www.dkrz.de/up/my-dkrz/getting-started/account/DKRZ-user-account). After that, you need to join a DKRZ project. For the group for IPCC related data analysis activities in the IPCC DDC Virtual Workspace and the IS-ENES3 related data analysis activities, as the [Analysis Platforms](https://portal.enes.org/data/data-metadata-service/analysis-platforms), the assigned project is bk1088. It means that the processing time (measured in CPU hours) and the storage (measured in gigabytes of memory) you use are allocated in that project.

We follow the original idea and code developed by Dr. Sebastian Milinski from the Max Planck Institute for Meteorology (MPI-M), a chapter scientist for Working Group 1 (chapter 4) of the IPCC AR6. Thanks to the DKRZ software developer Ralf Mueller for porting the notebook to the DKRZ Jupyterhub and the DKRZ data scientist Dr. Maria Moreno for writing the tutorial. Contact her for questions and comments at moreno@dkrz.de.

# Import the required python and cdo packages

All we need is available in the Python 3.5.2 module, choose this kernel (right upper corner).

In [ ]:
# Python packages 
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import netcdf
import pandas as pd 
import xarray as xr 
import os

# Import the python binder for the Climate Data Operators
import cdo
cdo = cdo.Cdo() 

# to render the figures in this notebook
%matplotlib inline 

# Data paths 

All available CMIP6 data at the DKRZ data pool are in: /work/ik1017/CMIP6/data/CMIP6

Check the CMIP6 global availability [here](https://pcmdi.llnl.gov/CMIP6/ArchiveStatistics/esgf_data_holdings/). Use the web accessible search index (one can use the "guest" option) [here](http://cmip-esmvaltool.dkrz.de). If you are missing data at the DKRZ data pool, we can make a replica on demand, write to data-pool@dkrz.de. 

In order to accelerate the data load, we computed global mean time series for some CMIP6 model results in these folders:

In [ ]:
datdir_hist='/work/bk1088/cmip6_selection/CMIP6_GSAT/CMIP6/historical/all/'
datdir_ssp245='/work/bk1088/cmip6_selection/CMIP6_GSAT/CMIP6/ssp245/all/'

# Identify the historical and scenario match
Note that some models don't have SSP2-4.5 simulations (yet). And for some models, the historical results might not yet be available on the data pool on Mistral. We will only plot time series for models where both the historical and SSP2-4.5 scenario are available.

In [ ]:
# for historical-------------------------------------------------------------
files_hist = os.listdir(datdir_hist)
files_hist.sort()

# filter for r1
subs = '_r1i1'

# using list comprehension to get string with substring  
r1_files_hist = [model for model in files_hist if subs in model] 
# print(r1_files_hist)

# extract model names
models = [filename.split('_')[2] for filename in r1_files_hist]
# remove duplicate names
unique_models = list(dict.fromkeys(models))

# identify highest version numer
modelversions_latest_hist = {}
for model in unique_models:
    model_files_hist = [filename for filename in r1_files_hist if model in filename]
#     print(model_files_hist)
    versions = [filename.split('_')[7] for filename in model_files_hist]
    versions.sort()
    modelversions_latest_hist[model]=versions[-1]
    
#print(modelversions_latest_hist)

# for ssp245------------------------------------------------------------------
files_ssp245 = os.listdir(datdir_ssp245)
files_ssp245.sort()

# filter for r1
subs = '_r1i1'

# to get string with substring  
r1_files_ssp245 = [model for model in files_ssp245 if subs in model] 

# extract model names
models = [filename.split('_')[2] for filename in r1_files_ssp245]
# remove duplicate names
unique_models = list(dict.fromkeys(models))

# identify highest version numer
modelversions_latest_ssp245 = {}
for model in unique_models:
    model_files_ssp245 = [filename for filename in r1_files_ssp245 if model in filename]
    versions = [filename.split('_')[7] for filename in model_files_ssp245]
    versions.sort()
    modelversions_latest_ssp245[model]=versions[-1]

#print(modelversions_latest_ssp245)

In [ ]:
# generate list of models that have historical AND ssp245 runs
model_list = []
for key in modelversions_latest_hist:
    if key in modelversions_latest_ssp245:
        model_list.append(key)

print(model_list)

# Load the data 

In [ ]:
# Historical

# initialise array:
operator = '-yearmean -mergetime'
ds_hist = []
loop = 0
for model in list(model_list):
    # find all files for latest version
    model_files = [filename for filename in r1_files_hist if model in filename]
    latest_model_files = [filename for filename in model_files if modelversions_latest_hist[model] in filename]
    ifile = datdir_hist + '*' + model + '*' + modelversions_latest_hist[model] + '*'
    model_timeseries = cdo.addc('0',input=operator+' '+ifile)
    model_timeseries = xr.open_dataset(model_timeseries)['tas'].squeeze()
    # add modelname as attribute
    model_timeseries.attrs['model'] = model
    # some models don't have a readable time axis (replacing with time series derived from first year and length of time axis)
    firstyear = str(model_timeseries['time'][0].values)[0:4]
    time_length = model_timeseries.size
    model_timeseries.coords['time'] = pd.date_range(firstyear + '-01-01', periods=time_length, freq='A-JUL')
    # some models have historical simulations beyond 2014
    model_timeseries = model_timeseries.loc['1850-01-01':'2014-12-31']
    # remove height coords if they exist. Causes problems with xr.concat
    if 'height' in model_timeseries.coords:
        del model_timeseries['height']


    ds_hist.append(model_timeseries)

In [ ]:
# Scenario ssp245

# initialise array:
operator = '-selyear,2015/2099 -yearmean -mergetime' # some models continue their scenario runs after 2099. Here, we only use output until 2099, which is available for all models.
ds_ssp245 = []
for model in list(model_list):
    # find all files for latest version
    model_files = [filename for filename in r1_files_ssp245 if model in filename]
    latest_model_files = [filename for filename in model_files if modelversions_latest_ssp245[model] in filename]
    ifile = datdir_ssp245 + '*' + model + '*' + modelversions_latest_ssp245[model] + '*'
    model_timeseries = cdo.addc('0',input=operator+' '+ifile)
    model_timeseries = xr.open_dataset(model_timeseries)['tas'].squeeze()
    # add modelname as attribute
    model_timeseries.attrs['model'] = model
    # some models don't have a readable time axis (replacing with time series derived from first year and length of time axis)
    model_timeseries.coords['time'] = pd.date_range('2015-01-01', periods=85, freq='A-JUL')
    # remove height coords if they exist. Causes problems with xr.concat
    if 'height' in model_timeseries.coords:
        del model_timeseries['height']

    ds_ssp245.append(model_timeseries)

In [ ]:
# create new dataset with concatenated time series
# last year of historical (2014) is added as a first value to the scenario timeseries to avoid a white gap 
# between 2014 and 2015 in the plot.
ds_merged_ssp245 = []

for i in list(range(0,len(ds_hist))):
    tas_array = np.concatenate((np.array([ds_hist[i].values[-1]]),ds_ssp245[i].values))
    time_array = np.concatenate((np.array([ds_hist[i]['time'].values[-1]]),ds_ssp245[i]['time'].values))
    timeseries = xr.DataArray(data=tas_array,coords=[time_array],dims='time')
    if ds_hist[i].attrs['model'] is ds_ssp245[i].attrs['model']:
        timeseries.attrs['model'] = ds_hist[i].attrs['model']
    else:
        print('model names do not match')
        
    ds_merged_ssp245.append(timeseries)

# Plot the multimodel comparison of the global mean annual mean near-surface air temperature SSP2-4.5 scenario

In [ ]:
# Plots path
plotdir = './plots/CMIP6_overview/'
if not os.path.exists(plotdir):
    os.makedirs(plotdir)
    
# IPCC colors
color_245=[155/255,135/255,12/255]
color_historical = [160/255,160/255,160/255]

# plotting options (choose reference period to compute anomalies)
refperiod_start = 1995
refperiod_end   = 2014

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(15,8))
plt.rcParams.update({'font.size': 15})

# Compare the annual mean of the MPI-ESM1-2-LR and the IPSL-CM6A-LR model results, for example
i=0
for array in ds_merged_ssp245:
    offset_hist=ds_hist[i].loc[str(refperiod_start)+'-01-01':str(refperiod_end)+'-12-31'].mean(dim='time')
    if array.attrs['model'] == 'MPI-ESM1-2-LR':
        axes.plot(ds_hist[i]['time'],ds_hist[i]-offset_hist, color='cyan',label='',zorder=10)
        axes.plot(array['time'],array-offset_hist, color='cyan',label=array.attrs['model'],zorder=10) 
        i+=1
    elif array.attrs['model'] == 'IPSL-CM6A-LR':
        axes.plot(ds_hist[i]['time'],ds_hist[i]-offset_hist, color='blue',label='',zorder=10) 
        axes.plot(array['time'],array-offset_hist, color='blue',label=array.attrs['model'],zorder=10) 
        i+=1
    else:
        axes.plot(ds_hist[i]['time'],ds_hist[i]-offset_hist, color=color_historical,label='') 
        axes.plot(array['time'],array-offset_hist, color=color_245,label='') 
        i+=1

# zeroline
x_pos1=np.datetime64('1850-01-01')
x_pos2=np.datetime64('2100-12-31')
axes.plot([x_pos1,x_pos2],[0,0], color='black',linewidth=1)

legend = axes.legend(loc='upper left', shadow=True, fontsize='large',ncol=1,frameon=False)
axes.set_xlim([pd.Timestamp('1850-01-01'),pd.Timestamp('2100-01-01')])
axes.set_ylim([-2,5])

plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True) # labels along the bottom edge are off

plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelleft=True) # labels along the bottom edge are off

# introduce second y-axis (makes it easier to read off warming in 2100)
ax2 = axes.twinx()
mn, mx = axes.get_ylim()
ax2.set_ylim(mn, mx)
ax2.set_ylabel('')

axes.set_ylabel('anomaly relative to '+str(refperiod_start)+'-'+str(refperiod_end)+' (K)')
axes.set_title('Global mean annual mean near-surface air temperature CMIP6 historical + SSP2-4.5')
plt.savefig(plotdir+'SSP2-4.5.pdf', bbox_extra_artists=(legend,), bbox_inches='tight', dpi=300)
plt.show()

See above that a .pdf with the figure is created in a folder in your Mistral home called "/plots/CMPI6_overview". You can download the plot to your local computer by writing "scp b123456@mistral:/home/dkrz/b123456/plots/CMIP6_overview/SSP2-4.5.pdf . " in your local folder, where the last "." means that the plot must be downloaded in the folder you are and "b123456" must be replaced by your actual user name.